**Testing for pairwise_colocalisation**

# Load data

In [2]:
import pandas as pd
from Sequential_Fish.tools import safe_merge_no_duplicates
from Sequential_Fish.analysis.post_processing import RNA_filtering, Spots_filtering

RUN_PATH = "/mnt/ssd/SSD_floricslimani/Fish_seq/Davide/2024-09-03 - SeqFISH_Run8 - HeLa_POLR2"

In [3]:
Acquisition = pd.read_feather(RUN_PATH + "/result_tables/Acquisition.feather")
Detection = pd.read_feather(RUN_PATH + "/result_tables/Detection.feather")
Spots = pd.read_feather(RUN_PATH + "/result_tables/Spots.feather")
Gene_map = pd.read_feather(RUN_PATH + "/result_tables/Gene_map.feather")
Cell = pd.read_feather(RUN_PATH + "/result_tables/Cell.feather")

COLOCALISATION_DISTANCE = 400


In [4]:
Cell = Cell.rename(columns={'label' : 'cell_label', 'bbox' : 'cell_bbox'})
Cell

,quantification_id,index_mean_distance_cell,index_median_distance_cell,index_mean_distance_nuc,index_median_distance_nuc,proportion_rna_in_nuc,nb_rna_out_nuc,nb_rna_in_nuc,index_rna_protrusion,proportion_rna_protrusion,...,nucleus_mean_mean_signal,nucleus_mean_max_signal,nucleus_mean_min_signal,nucleus_mean_median_signal,acquisition_id,detection_id,cluster_number,rna_number,location,cell_id
0,0,0.92,0.85,1.49,2.58,0.15,134.0,24.0,0.0,0.0,...,4609.652813,5953,2472,4664.5,0,1,8,158,Location-01,0
1,1,1.00,1.00,1.00,1.00,0.00,0.0,0.0,1.0,0.0,...,4609.652813,5953,2472,4664.5,10,2,0,0,Location-01,0
2,2,0.89,0.86,1.15,1.86,0.24,90.0,29.0,0.0,0.0,...,4609.652813,5953,2472,4664.5,20,3,0,119,Location-01,0
3,3,1.00,1.00,1.00,1.00,0.00,0.0,0.0,1.0,0.0,...,4609.652813,5953,2472,4664.5,30,4,0,0,Location-01,0
4,4,1.13,1.07,1.16,1.81,0.24,75.0,24.0,0.0,0.0,...,4609.652813,5953,2472,4664.5,40,5,0,99,Location-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6652,6652,1.23,1.46,0.38,0.00,0.58,16.0,22.0,0.0,0.0,...,7109.708382,9057,3908,7277.5,169,206,0,38,Location-10,316
6653,6653,1.00,1.00,1.00,1.00,0.00,0.0,0.0,1.0,0.0,...,7109.708382,9057,3908,7277.5,179,207,0,0,Location-10,316
6654,6654,1.15,1.29,0.48,0.11,0.47,27.0,24.0,0.0,0.0,...,7109.708382,9057,3908,7277.5,189,208,0,51,Location-10,316
6655,6655,1.00,1.00,1.00,1.00,0.00,0.0,0.0,1.0,0.0,...,7109.708382,9057,3908,7277.5,199,209,0,0,Location-10,316


In [5]:
Detection = safe_merge_no_duplicates(
    Detection,
    Acquisition,
    on= ['acquisition_id'],
    keys=['cycle','location', 'fish_reodered_shape']
)

Detection = safe_merge_no_duplicates(
    Detection,
    Gene_map,
    on= ['cycle','color_id'],
    keys=['target']
)

Spots =safe_merge_no_duplicates(
    Spots,
    Detection,
    on= 'detection_id',
    keys= ['location','target', 'voxel_size', 'fish_reodered_shape']
)


Spots

/home/floric/miniconda3/envs/dev/lib/python3.8/site-packages/Sequential_Fish/tools/utils.py:154: UserWarning: location already in left dataframe columns, location was removed from columns to merge.
  warnings.warn(f"{key} already in left dataframe columns, {key} was removed from columns to merge.")


,spot_id,cluster_id,drifted_z,drifted_y,drifted_x,intensity,population,detection_id,acquisition_id,drift_z,...,cycle,color_id,is_washout,coordinates,in_nucleus,cell_label,location,target,voxel_size,fish_reodered_shape
0,0,NaN,5,0,597,5927,free,1,0,0,...,0,0,False,"[5, 0, 597]",True,2.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
1,1,NaN,5,77,389,6095,free,1,0,0,...,0,0,False,"[5, 77, 389]",False,1.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
2,2,NaN,5,141,993,6923,free,1,0,0,...,0,0,False,"[5, 141, 993]",False,5.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
3,3,NaN,5,144,1107,6960,free,1,0,0,...,0,0,False,"[5, 144, 1107]",True,5.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
4,4,NaN,5,265,673,7111,free,1,0,0,...,0,0,False,"[5, 265, 673]",True,6.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338989,288079,NaN,11,812,174,8032,free,174,58,0,...,5,0,True,"[11, 812, 174]",True,16.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"
338990,288080,0.0,5,1218,755,17217,clustered,174,58,0,...,5,0,True,"[5, 1218, 755]",True,28.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"
338991,288081,0.0,5,1217,757,16242,clustered,174,58,0,...,5,0,True,"[5, 1217, 757]",True,28.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"
338992,288082,0.0,5,1217,755,16993,clustered,174,58,0,...,5,0,True,"[5, 1217, 755]",True,28.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"


In [6]:
Spots = RNA_filtering(Spots)
Spots = Spots_filtering(Spots)
Spots = Spots.loc[~Spots['target'].str.contains('Washout')]
Spots

TypeError: Spots_filtering() missing 1 required positional argument: 'Cell'

## Rename

In [ ]:
rule = {
    'POLR2A_0' : 'POLR2A'
}
Spots['target'] = Spots['target'].replace(rule)

# NearestNeighbours

In [ ]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm #loading bar
import numpy as np

#We test colocalisation of population 1 with population 2, ie population 2 is used to fit neighbours model
POPULATION_1 = 'all'
POPULATION_2 = 'all'

In [ ]:
Spots

,spot_id,cluster_id,drifted_z,drifted_y,drifted_x,intensity,population,detection_id,acquisition_id,drift_z,...,cycle,color_id,is_washout,coordinates,in_nucleus,cell_label,location,target,voxel_size,fish_reodered_shape
0,0,NaN,5,0,597,5927,free,1,0,0,...,0,0,False,"[5, 0, 597]",True,2.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
1,1,NaN,5,77,389,6095,free,1,0,0,...,0,0,False,"[5, 77, 389]",False,1.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
2,2,NaN,5,141,993,6923,free,1,0,0,...,0,0,False,"[5, 141, 993]",False,5.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
3,3,NaN,5,144,1107,6960,free,1,0,0,...,0,0,False,"[5, 144, 1107]",True,5.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
4,4,NaN,5,265,673,7111,free,1,0,0,...,0,0,False,"[5, 265, 673]",True,6.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338678,338986,NaN,47,867,1705,4061,free,208,189,0,...,18,0,False,"[47, 867, 1705]",False,19.0,Location-10,POLR2L,"[200, 97, 97]","[61, 2004, 2004, 2]"
338680,338988,NaN,47,1463,583,4628,free,208,189,0,...,18,0,False,"[47, 1463, 583]",True,27.0,Location-10,POLR2L,"[200, 97, 97]","[61, 2004, 2004, 2]"
338682,338990,NaN,47,1656,934,4044,free,208,189,0,...,18,0,False,"[47, 1656, 934]",False,37.0,Location-10,POLR2L,"[200, 97, 97]","[61, 2004, 2004, 2]"
338683,338991,NaN,48,1786,1581,4024,free,208,189,0,...,18,0,False,"[48, 1786, 1581]",False,39.0,Location-10,POLR2L,"[200, 97, 97]","[61, 2004, 2004, 2]"


In [ ]:
def check_unique(df: pd.DataFrame, key :str) :
    
    if df[key].dtype == object :
        df[key] = df[key].apply(tuple)
    
    assert len(df[key].unique()) == 1, f"{key} is not unique for dataframe. \n{df[key].unique()}"

In [ ]:
# def add_random_spots_from_population(Spots : pd.DataFrame) :
    # """
    # Add random spots for coloc rates normalisation.
    # """
    
Spots['is_random'] = False
max_spot_id = Spots['spot_id'].max()
random_generator = np.random.default_rng(seed=1)

counts = Spots.groupby(['target','location'])['spot_id'].count()
print(counts)

check_unique(Spots, 'voxel_size')
voxel_size = Spots.iloc[0].at['voxel_size']
check_unique(Spots, 'fish_reodered_shape')
shape = Spots.iloc[0].at['fish_reodered_shape'][:3]

targets = counts.index.get_level_values(0)
locations = counts.index.get_level_values(1)

for target, location, spot_number in zip(targets, locations, counts) :
    
    coordinates = random_generator.integers(
        low= (0,0,0),
        high=shape,
        size= (spot_number,3)
    )

    z,y,x = zip(*coordinates)
    spots_id = np.arange(max_spot_id + 1, max_spot_id + spot_number + 1)
    
    fake_spots = pd.DataFrame({
        'spot_id' : spots_id,
        'z' : z,
        'y' : y,
        'x' : x,
        'target' : [target] * spot_number,
        'location' : [location] * spot_number,
        'is_random' : [True] * spot_number,
        'voxel_size' : [voxel_size] * spot_number
    })
    
    Spots = pd.concat([
        Spots,
        fake_spots
    ], axis=0).reset_index(drop=True)

Spots

target  location   
POLR2A  Location-01    4416
        Location-02    6739
        Location-03    9953
        Location-04    8818
        Location-05    6969
                       ... 
POLR2L  Location-06    1334
        Location-07    1110
        Location-08    1097
        Location-09     917
        Location-10     946
Name: spot_id, Length: 100, dtype: int64


AssertionError: fish_reodered_shape is not unique for dataframe. 
[(51, 2004, 2004, 2) (56, 2004, 2004, 2) (61, 2004, 2004, 2)]

In [ ]:
def get_population_index(Spots : pd.DataFrame, population_key:str) :
    """
    get index from Spots data frame to a population : 'all', 'clustered', 'free'
    """
    if population_key == 'all' : population_index = Spots.index
    elif population_key == 'clustered' : population_index = Spots.loc[~Spots['cluster_id'].isna()].index
    elif population_key == 'free' : population_index = Spots.loc[Spots['cluster_id'].isna()].index
    else : raise AssertionError("{} incorect key for population_key".format(population_key))

    return population_index

In [ ]:
def create_coordinate_df(Spots : pd.DataFrame, population_key:str) :
    
    population_index = get_population_index(Spots, population_key)
    coordinates_df = Spots.loc[population_index]

    #converting pixel coordinates to nanometers
    coordinates_df['voxel_size_z'], coordinates_df['voxel_size_y'], coordinates_df['voxel_size_x'] = list(zip(*coordinates_df['voxel_size']))
    coordinates_df['z'] *= coordinates_df['voxel_size_z'] 
    coordinates_df['y'] *= coordinates_df['voxel_size_y'] 
    coordinates_df['x'] *= coordinates_df['voxel_size_x'] 

    coordinates_df['coordinates'] = list(zip(coordinates_df['z'], coordinates_df['y'], coordinates_df['x']))
    coordinates_df = coordinates_df.groupby(['location','target'])['coordinates'].apply(list)
    return coordinates_df

In [ ]:
def create_neihbor_model_dict(spots_coordinates_df, colocalisation_distance) :
    neighbor_models_dict = dict()
    for idx in tqdm(spots_coordinates_df.index, desc= "Fitting nearest neighbors models") :
        spot_distribution = spots_coordinates_df.at[idx]
        new_model = NearestNeighbors(radius=colocalisation_distance)
        new_model.fit(spot_distribution)
        neighbor_models_dict[idx] = new_model
        
    return neighbor_models_dict

In [ ]:
def _colocalisation_truth_df(spots_coordinates_df : pd.DataFrame, Spots : pd.DataFrame, neighbor_models_dict : dict) :
    population1_index = get_population_index(Spots, population_key=POPULATION_1)
    RNAs = list(spots_coordinates_df.index.get_level_values(1).unique())
    colocalisation_truth_df = pd.DataFrame(index=population1_index, columns= RNAs, dtype=bool)
    colocalisation_truth_df = colocalisation_truth_df.join(Spots.loc[:,['spot_id','location','target', 'z','y','x','voxel_size']])

    #converting coordinates to nanometers
    colocalisation_truth_df['voxel_size_z'], colocalisation_truth_df['voxel_size_y'], colocalisation_truth_df['voxel_size_x'] = list(zip(*colocalisation_truth_df['voxel_size']))
    colocalisation_truth_df['z'] *= colocalisation_truth_df['voxel_size_z'] 
    colocalisation_truth_df['y'] *= colocalisation_truth_df['voxel_size_y'] 
    colocalisation_truth_df['x'] *= colocalisation_truth_df['voxel_size_x'] 
    colocalisation_truth_df['coordinates'] = list(zip(colocalisation_truth_df['z'], colocalisation_truth_df['y'], colocalisation_truth_df['x']))

    colocalisation_truth_df = colocalisation_truth_df.drop(columns=['z','y','x','voxel_size','voxel_size_z','voxel_size_y','voxel_size_x'])
    
    for location in tqdm(colocalisation_truth_df['location'].unique(), desc= "computing co-localizations") :
        target_idx = colocalisation_truth_df[colocalisation_truth_df['location'] == location].index
        for rna in RNAs :
            model : NearestNeighbors = neighbor_models_dict[(location, rna)]
            coordinates = list(colocalisation_truth_df.loc[target_idx]['coordinates'].apply(np.array,dtype=int))
            coordinates = np.array(coordinates, dtype=int)
            query = model.radius_neighbors(coordinates, return_distance=False)
            query = pd.Series(query, index=target_idx).apply(len).apply(bool) #if count is 0 no colocalisation -> False else True
            colocalisation_truth_df.loc[target_idx,[rna]] = query
    
    return colocalisation_truth_df

In [ ]:
def colocalisation_truth_df(
    Spots : pd.DataFrame,
    population_1 = 'all',
    population_2 = 'all',
    colocalisation_distance = 400,
) :
    
    population_1_index = get_population_index(Spots, population_key=population_1)
    
    real_spots_coordinates_df = create_coordinate_df(Spots.loc[~Spots['is_random']], population_key= population_2)
    random_spots_coordinates_df = create_coordinate_df(Spots.loc[Spots['is_random']], population_key= population_2)
    
    neighbor_models_dict = create_neihbor_model_dict(real_spots_coordinates_df, colocalisation_distance=colocalisation_distance)
    real_colocalisation_truth_df = _colocalisation_truth_df(real_spots_coordinates_df, Spots.loc[population_1_index], neighbor_models_dict)
    random_colocalisation_truth_df = _colocalisation_truth_df(random_spots_coordinates_df, Spots, neighbor_models_dict)
    
    return real_colocalisation_truth_df, random_colocalisation_truth_df

In [ ]:
colocalisation_truth_df, random_colocalisation_truth_df = colocalisation_truth_df(
    Spots,
    population_1= 'all',
    population_2= 'all',
    colocalisation_distance=COLOCALISATION_DISTANCE
)
colocalisation_truth_df

computing co-localizations: 100%|██████████| 10/10 [00:15<00:00,  1.58s/it]


,POLR2A,POLR2B,POLR2C,POLR2E,POLR2F,POLR2H,POLR2I,POLR2J,POLR2K,POLR2L,spot_id,location,target,coordinates
0,True,False,False,False,False,False,False,False,False,False,1,Location-01,POLR2A,"(1000, 291, 141911)"
1,True,False,False,False,False,False,False,False,False,False,5,Location-01,POLR2A,"(1000, 970, 148895)"
2,True,False,False,False,False,False,False,False,False,False,7,Location-01,POLR2A,"(1000, 1649, 139971)"
3,True,False,False,False,False,False,False,False,False,False,8,Location-01,POLR2A,"(1000, 1843, 124451)"
4,True,False,False,False,False,True,False,False,False,False,10,Location-01,POLR2A,"(1000, 2328, 141911)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862857,False,False,False,False,False,False,False,False,False,False,572641,Location-10,POLR2L,"(8600, 186434, 188471)"
862858,False,False,False,False,False,False,False,False,False,False,572642,Location-10,POLR2L,"(5800, 14938, 168489)"
862859,False,False,False,False,False,False,False,False,False,False,572643,Location-10,POLR2L,"(1600, 25705, 23086)"
862860,False,False,False,False,False,False,False,False,False,False,572644,Location-10,POLR2L,"(10600, 116788, 62856)"


In [ ]:
random_colocalisation_truth_df

,POLR2A,POLR2B,POLR2C,POLR2E,POLR2F,POLR2H,POLR2I,POLR2J,POLR2K,POLR2L,spot_id,location,target,coordinates
0,True,False,False,False,False,False,False,False,False,False,1,Location-01,POLR2A,"(1000, 291, 141911)"
1,True,False,False,False,False,False,False,False,False,False,5,Location-01,POLR2A,"(1000, 970, 148895)"
2,True,False,False,False,False,False,False,False,False,False,7,Location-01,POLR2A,"(1000, 1649, 139971)"
3,True,False,False,False,False,False,False,False,False,False,8,Location-01,POLR2A,"(1000, 1843, 124451)"
4,True,False,False,False,False,True,False,False,False,False,10,Location-01,POLR2A,"(1000, 2328, 141911)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862857,False,False,False,False,False,False,False,False,False,False,572641,Location-10,POLR2L,"(8600, 186434, 188471)"
862858,False,False,False,False,False,False,False,False,False,False,572642,Location-10,POLR2L,"(5800, 14938, 168489)"
862859,False,False,False,False,False,False,False,False,False,False,572643,Location-10,POLR2L,"(1600, 25705, 23086)"
862860,False,False,False,False,False,False,False,False,False,False,572644,Location-10,POLR2L,"(10600, 116788, 62856)"


*** Error number of spots was doubled : len(Spots) shoud equal len(random_colocalisation_df) == real_colocalisation_df

# Plots

In [ ]:
def create_coloc_rates_df(Spots, colocalisation_truth_df) :
    RNA_list = list(Spots['target'].unique())
    RNA_list.sort()
    coloc_rates_df = pd.DataFrame(columns=RNA_list, index= RNA_list)

    for target in RNA_list :
        data = colocalisation_truth_df.loc[colocalisation_truth_df['target'] == target, RNA_list]
        spots_number = len(data)
        data = data.sum(axis=0)/spots_number*100
        coloc_rates_df[target] = data

    return coloc_rates_df

In [ ]:
coloc_rates_df = create_coloc_rates_df(Spots, colocalisation_truth_df)
coloc_rates_df

,POLR2A,POLR2B,POLR2C,POLR2E,POLR2F,POLR2H,POLR2I,POLR2J,POLR2K,POLR2L
POLR2A,50.314909,2.019915,2.065184,2.086462,1.981954,2.057883,2.120297,2.116189,2.541179,2.192394
POLR2B,0.327695,50.071124,0.812850,0.510447,0.363456,0.492578,0.298633,0.404069,0.354083,0.364206
POLR2C,0.511905,0.995733,50.054449,0.742381,0.590494,0.575770,0.483083,0.604470,0.504715,0.493065
POLR2E,1.068323,1.476529,1.542082,50.139731,1.325200,1.412058,1.050487,1.167552,1.175711,1.094407
POLR2F,0.958934,1.149360,1.108432,1.230965,50.146162,1.565305,0.871307,0.966063,0.944873,0.950336
POLR2H,0.440873,0.546230,0.528936,0.524705,0.510592,50.059109,0.372413,0.588133,0.496890,0.542282
POLR2I,0.622715,0.477952,0.526991,0.564628,0.472590,0.453172,50.068510,0.596846,0.854885,0.642506
POLR2J,0.908738,0.935989,1.001478,0.932492,0.848713,0.917291,0.957383,50.140498,0.929223,1.091723
POLR2K,0.742996,0.551920,0.521157,0.551320,0.489155,0.562634,0.693883,0.556548,50.064557,0.588814
POLR2L,1.215597,1.166430,1.141490,1.168229,1.060160,1.352949,1.377227,1.350527,1.361556,50.151230


# Sandbox for random spots colocalization testing

We want to compute random colocalization rates between populations with equal number of spot and equal volume.

Real spot population is bound by the cell in 3D, also, for some population they can be bound in or out the nucleus. Which is why we can't generate a random population in the bounding box of the cell and test it with the real population.

What we can try is to generate 2 random populations and test them together.
We can generate the populations in a rectangle of same area than the cell's area by computing the average number of spots/plane(1st approximation)
To account for 3D colocalization we can generate 3 planes and measure the colocalization  of the one in the middle. (3 be cause colocalization cannot happen further than 1 plane away)

**Steps :**
* compute for each cell and each population the average number of spot/plane (in the center of the cell) (**tends to increase concentration**)
* For each cell get area --> Compute size of a rectangle of same area
* Compute 3 random spots distributions in this area and assign z coordinates 0, 1, 2
* compute colocalization rate for z = 1
* plot distribution on all cells

## Load a spot table

In [39]:
import pandas as pd
from Sequential_Fish.tools import safe_merge_no_duplicates

RUN_PATH = "/mnt/ssd/SSD_floricslimani/Fish_seq/Davide/2024-09-03 - SeqFISH_Run8 - HeLa_POLR2"
Acquisition = pd.read_feather(RUN_PATH + "/result_tables/Acquisition.feather")
Detection = pd.read_feather(RUN_PATH + "/result_tables/Detection.feather")
Spots = pd.read_feather(RUN_PATH + "/result_tables/Spots.feather")
Gene_map = pd.read_feather(RUN_PATH + "/result_tables/Gene_map.feather")
Cell = pd.read_feather(RUN_PATH + "/result_tables/Cell.feather")


In [42]:
Detection = safe_merge_no_duplicates(
    Detection,
    Acquisition,
    on= ['acquisition_id'],
    keys=['cycle','location', 'fish_reodered_shape']
)

Detection = safe_merge_no_duplicates(
    Detection,
    Gene_map,
    on= ['cycle','color_id'],
    keys=['target']
)

Spots =safe_merge_no_duplicates(
    Spots,
    Detection,
    on= 'detection_id',
    keys= ['location','target', 'voxel_size', 'fish_reodered_shape']
)


Spots

/home/floric/miniconda3/envs/dev/lib/python3.8/site-packages/Sequential_Fish/tools/utils.py:154: UserWarning: location already in left dataframe columns, location was removed from columns to merge.
  warnings.warn(f"{key} already in left dataframe columns, {key} was removed from columns to merge.")


,spot_id,cluster_id,drifted_z,drifted_y,drifted_x,intensity,population,detection_id,acquisition_id,drift_z,...,cycle,color_id,is_washout,coordinates,in_nucleus,cell_label,location,target,voxel_size,fish_reodered_shape
0,0,NaN,5,0,597,5927,free,1,0,0,...,0,0,False,"[5, 0, 597]",True,2.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
1,1,NaN,5,77,389,6095,free,1,0,0,...,0,0,False,"[5, 77, 389]",False,1.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
2,2,NaN,5,141,993,6923,free,1,0,0,...,0,0,False,"[5, 141, 993]",False,5.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
3,3,NaN,5,144,1107,6960,free,1,0,0,...,0,0,False,"[5, 144, 1107]",True,5.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
4,4,NaN,5,265,673,7111,free,1,0,0,...,0,0,False,"[5, 265, 673]",True,6.0,Location-01,POLR2A,"[200, 97, 97]","[51, 2004, 2004, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338989,288079,NaN,11,812,174,8032,free,174,58,0,...,5,0,True,"[11, 812, 174]",True,16.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"
338990,288080,0.0,5,1218,755,17217,clustered,174,58,0,...,5,0,True,"[5, 1218, 755]",True,28.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"
338991,288081,0.0,5,1217,757,16242,clustered,174,58,0,...,5,0,True,"[5, 1217, 757]",True,28.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"
338992,288082,0.0,5,1217,755,16993,clustered,174,58,0,...,5,0,True,"[5, 1217, 755]",True,28.0,Location-09,Washout_5_0,"[200, 97, 97]","[56, 2004, 2004, 2]"


### Filtering spots

In [43]:
from Sequential_Fish.analysis.post_processing import RNA_filtering, Spots_filtering

Spots = RNA_filtering(Spots)
Spots = Spots_filtering(Spots, Cell=Cell)
Spots = Spots.loc[~Spots['target'].str.contains('Washout')]
Spots = Spots.loc[~Spots['target'].str.contains('Washout')]
Spots

,spot_id,cluster_id,drifted_z,drifted_y,drifted_x,intensity,population,detection_id,acquisition_id_x,drift_z,...,nucleus_mip_min_signal,nucleus_mip_median_signal,nucleus_mean_mean_signal,nucleus_mean_max_signal,nucleus_mean_min_signal,nucleus_mean_median_signal,acquisition_id_y,cluster_number,rna_number,cell_id
0,2,NaN,5,141,993,6923,free,1,0,0,...,2869,5519.0,4609.652813,5953,2472,4664.5,0,8,158,0
1,3,NaN,5,144,1107,6960,free,1,0,0,...,2869,5519.0,4609.652813,5953,2472,4664.5,0,8,158,0
2,78,NaN,7,166,1097,6533,free,1,0,0,...,2869,5519.0,4609.652813,5953,2472,4664.5,0,8,158,0
3,115,NaN,9,148,1035,7139,free,1,0,0,...,2869,5519.0,4609.652813,5953,2472,4664.5,0,8,158,0
4,211,NaN,14,78,1076,6400,free,1,0,0,...,2869,5519.0,4609.652813,5953,2472,4664.5,0,8,158,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268373,338723,NaN,35,946,145,4862,free,208,189,0,...,3480,5905.0,4797.449686,6468,2860,4851.5,189,0,14,299
268374,338724,NaN,33,947,144,4859,free,208,189,0,...,3480,5905.0,4797.449686,6468,2860,4851.5,189,0,14,299
268375,338725,NaN,34,973,276,4243,free,208,189,0,...,3480,5905.0,4797.449686,6468,2860,4851.5,189,0,14,299
268376,338770,NaN,34,988,128,4261,free,208,189,0,...,3480,5905.0,4797.449686,6468,2860,4851.5,189,0,14,299


## Group spots and get number of spots per plane

In [44]:
grouped_spots = Spots.groupby(['location','cell_label','detection_id','target','z'], as_index=False).agg({
    'spot_id' : 'count',
})

grouped_spots = grouped_spots.rename(columns={"spot_id" : 'spot_count'})
grouped_spots


,location,cell_label,detection_id,target,z,spot_count
0,Location-01,5.0,1,POLR2A,5,3
1,Location-01,5.0,1,POLR2A,7,2
2,Location-01,5.0,1,POLR2A,9,3
3,Location-01,5.0,1,POLR2A,10,3
4,Location-01,5.0,1,POLR2A,11,3
...,...,...,...,...,...,...
80932,Location-10,40.0,208,POLR2L,35,2
80933,Location-10,40.0,208,POLR2L,37,2
80934,Location-10,40.0,208,POLR2L,38,1
80935,Location-10,40.0,208,POLR2L,39,2


In [45]:
check_len = len(grouped_spots)

if 'cell_area' not in grouped_spots.columns :
    grouped_spots = pd.merge(
        grouped_spots,
        Cell.loc[:,['location','label', 'detection_id','cell_area']],
        left_on=['location','cell_label', 'detection_id'],
        right_on=['location','label', 'detection_id'],
    )

assert len(grouped_spots) == check_len, f"len {len(grouped_spots)} : check : {check_len}"
grouped_spots

,location,cell_label,detection_id,target,z,spot_count,label,cell_area
0,Location-01,5.0,1,POLR2A,5,3,5,56440.0
1,Location-01,5.0,1,POLR2A,7,2,5,56440.0
2,Location-01,5.0,1,POLR2A,9,3,5,56440.0
3,Location-01,5.0,1,POLR2A,10,3,5,56440.0
4,Location-01,5.0,1,POLR2A,11,3,5,56440.0
...,...,...,...,...,...,...,...,...
80932,Location-10,40.0,208,POLR2L,35,2,40,46475.0
80933,Location-10,40.0,208,POLR2L,37,2,40,46475.0
80934,Location-10,40.0,208,POLR2L,38,1,40,46475.0
80935,Location-10,40.0,208,POLR2L,39,2,40,46475.0


In [69]:
BOTTOM = 5
PLANE_NUMBER = 20

total_per_plane = grouped_spots[(grouped_spots['z'] >= BOTTOM) & (grouped_spots['z'] < BOTTOM + PLANE_NUMBER)].groupby(['location','cell_label','detection_id','target'])['spot_count'].sum()
average_per_plane = (total_per_plane/PLANE_NUMBER).rename('spot_per_plane').reset_index(drop=False)


single_per_plane = average_per_plane.groupby(['target'])['spot_per_plane'].mean()
single_per_plane

target
POLR2A    4.039590
POLR2B    1.907886
POLR2C    1.893060
POLR2E    2.632965
POLR2F    2.395426
POLR2H    1.013133
POLR2I    0.949051
POLR2J    1.555836
POLR2K    0.430420
POLR2L    0.933121
Name: spot_per_plane, dtype: float64

In [ ]:
def create_plane_with_random_distribution()